# Assumptions:
- Zero interest rates
- No transport delays
- Injection and withdrawal costs are per MMBtu
- Injection/withdrawal rates are per event
- Storage cost is charged monthly on maximum storage used
- Prices are obtained from an estimated forward curve
`` providing a fair deterministic estimate of contract value.


In [5]:
#import dependencies
import pandas as pd
from datetime import datetime
from price_utils import estimate_gas_price,build_price_curve
df=pd.read_csv("Nat_Gas.csv")
combined_df = build_price_curve(df)

In [6]:
def price_storage_contract(
    injection_events,
    withdrawal_events,
    injection_rate,
    withdrawal_rate,
    max_storage,
    storage_cost_per_month,
    injection_cost_per_unit,
    withdrawal_cost_per_unit,
    combined_df
):
    """
    Prices a natural gas storage contract using deterministic cashflows
    """

    # Combine all events
    events = []

    for inj in injection_events:
        events.append({
            "date": inj["date"],
            "volume": inj["volume"],
            "type": "inject"
        })

    for wd in withdrawal_events:
        events.append({
            "date": wd["date"],
            "volume": wd["volume"],
            "type": "withdraw"
        })

    # Sort events chronologically
    events.sort(key=lambda x: x["date"])

    # State variables
    current_storage = 0
    max_storage_used = 0
    total_cashflow = 0

    # Process events
    for event in events:
        date = event["date"]
        requested_volume = event["volume"]
        price = estimate_gas_price(date, combined_df)

        # Injection event
        if event["type"] == "inject":
            allowed_volume = min(
                requested_volume,
                injection_rate,
                max_storage - current_storage
            )

            total_cashflow -= price * allowed_volume
            total_cashflow -= injection_cost_per_unit * allowed_volume

            current_storage += allowed_volume
            max_storage_used = max(max_storage_used, current_storage)

        # Withdrawal event
        elif event["type"] == "withdraw":
            allowed_volume = min(
                requested_volume,
                withdrawal_rate,
                current_storage
            )

            total_cashflow += price * allowed_volume
            total_cashflow -= withdrawal_cost_per_unit * allowed_volume

            current_storage -= allowed_volume

    # Storage cost calculation
    all_dates = [e["date"] for e in events]
    start_date = datetime.strptime(min(all_dates), "%Y-%m-%d")
    end_date = datetime.strptime(max(all_dates), "%Y-%m-%d")

    duration_months = (
        (end_date.year - start_date.year) * 12
        + (end_date.month - start_date.month)
    )

    storage_cost = duration_months * max_storage_used * storage_cost_per_month
    total_cashflow -= storage_cost

    return total_cashflow


In [10]:
#examples to test
injection_events = [
    {"date": "2024-04-01", "volume": 500_000},
    {"date": "2024-07-01", "volume": 300_000},
]

withdrawal_events = [
    {"date": "2025-01-01", "volume": 400_000},
    {"date": "2025-02-01", "volume": 400_000},
]

contract_value = price_storage_contract(
    injection_events=injection_events,
    withdrawal_events=withdrawal_events,
    injection_rate=500_000,
    withdrawal_rate=500_000,
    max_storage=1_000_000,
    storage_cost_per_month=0.02,
    injection_cost_per_unit=0.01,
    withdrawal_cost_per_unit=0.01,
    combined_df=combined_df
)

print("Estimated storage contract value:", contract_value)


Estimated storage contract value: 115316.5276856646
